These are TODO's from the EDA section to be implemented in the present notebook.
- FEATURE: replace zipcodes with respective mean longitude and latitude
- FEATURE: area of each area could be added, along with density of panels for the given zip code 
- FEATURE: approximation of zipcode by first 2 digits 
- FEATURE: add aditional feature to mark US and non-US states

- NULLS: drop nulls 

- NORMALISATION: scaling num of panels group of features is required. Simply dividing by a constant to bring the mean of distribution closer to other features or normalising by mean could be attempted

# Table of Contents
- **[Feature extraction](#1)** 
- **[Nulls](#2)** 
- **[Normalisation](#3)** 

In [1]:
import numpy as np
import pandas as pd

In [80]:
init_df = pd.read_csv('datasets/geo_solar_panels.csv')
ref_df =  pd.read_csv('datasets/zip_code_database.csv')[['zip', 'state', 'latitude', 'longitude']]
state_accronyms = pd.read_csv('datasets/states_accro_mapping.csv', sep=';')

# Feature extraction

In [22]:
class FeatureExtractor:
    def __init__(self, long_min: pd.Series, long_max: pd.Series, lat_min: pd.Series, lat_max: pd.Series,
                 num_of_panels: pd.Series, usa_states: tuple[str], states_to_check: pd.Series,
                 lat_long_zip_df: pd.DataFrame, ref_df: pd.DataFrame, init_df_zip_col: str, ref_df_zip_col: str, 
                 init_df_mean_lat_col: str, init_df_mean_long_col: str, ref_df_lat_col: str, ref_df_long_col: str
                 ):
        self.long_min = long_min
        self.long_max = long_max
        self.lat_min = lat_min
        self.lat_max = lat_max

        self.num_of_panels = num_of_panels
        self.usa_states = usa_states
        self.states_to_check = states_to_check
        
        self.lat_long_zip_df = lat_long_zip_df
        self.ref_df = ref_df
        self.init_df_zip_col = init_df_zip_col
        self.ref_df_zip_col = ref_df_zip_col
        self.init_df_mean_lat_col = init_df_mean_lat_col
        self.init_df_mean_long_col = init_df_mean_long_col
        self.ref_df_lat_col = ref_df_lat_col
        self.ref_df_long_col = ref_df_long_col

    def calc_mean_coords(self) -> tuple[pd.Series, pd.Series]:
        """Function calculates mean coordinates for a zip code"""
        lat_mean = (self.lat_max + self.lat_min) / 2
        long_mean = (self.long_max + self.long_min) / 2
        return (lat_mean, long_mean)

    def calc_zip_area(self) -> pd.Series:
        """The function calculates approx. area of zip code using min/max longitude and lattitude"""
        # adding small number to avoid zero area
        return abs(self.long_max - self.long_min) * abs(self.lat_max - self.lat_min) + 1e-5
    
    def panels_density(self, zip_area: pd.Series) -> pd.Series:
        """Function calculates panels density in a zip_code based on zip_code area and number of panels for the zip_code"""
        return self.num_of_panels / zip_area
    
    def approx_zips(self) -> pd.Series:
        """Function returns a column with first 2 digits of zipcode given a columns of zipcodes"""
        return self.lat_long_zip_df[self.init_df_zip_col].astype('str').apply(lambda x: int(x[:2]))
    
    def mask_usa_states(self) -> pd.Series:
        """Function checks if states in the passed column are USA states"""
        return self.states_to_check.isin(self.usa_states)
    
    def zips_replacement(self):
        """
        Function merged reference df to the provided df with lat and long and returns mean lat and long with the same index. 
        For zips with no records in the reference df, values are filled with mean coordinates from the original df.
        """
        local_df = self.lat_long_zip_df.merge(ref_df, how='left', left_on=self.init_df_zip_col, right_on=self.ref_df_zip_col)
        zip_lat = local_df[self.ref_df_lat_col].fillna(value=local_df[self.init_df_mean_lat_col])
        zip_long = local_df[self.ref_df_long_col].fillna(value=local_df[self.init_df_mean_long_col])
        return zip_lat, zip_long

# Nulls

# Normalisation